In [1]:
import parl
from parl import layers
from parl.utils import logger
from parl.utils import action_mapping  # 将神经网络输出映射到对应的 实际动作取值范围内
from parl.utils import ReplayMemory  # 经验回放

from td3_model import RLBenchModel
from td3_agent import RLBenchAgent
from parl.algorithms import TD3
import gym
import rlbench.gym
import numpy as np

/home/wang/venv_parl/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
env = gym.make('reach_target-state-v0', render_mode='human')
env.reset()
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

print('act_dim: {}, obs_dim: {}'.format(act_dim, obs_dim))

2020-06-30 14:40:20,527-INFO: Resetting task: reach_target
2020-06-30 14:40:20,725-INFO: Resetting task: reach_target
act_dim: 8, obs_dim: 40


In [3]:
GAMMA = 0.99  # reward 的衰减因子，一般取 0.9 到 0.999 不等
TAU = 0.001  # target_model 跟 model 同步参数 的 软更新参数
ACTOR_LR = 0.0002  # Actor网络更新的 learning rate
CRITIC_LR = 0.001  # Critic网络更新的 learning rate
MEMORY_SIZE = 1e6  # replay memory的大小，越大越占用内存
MEMORY_WARMUP_SIZE = 1e3  # replay_memory 里需要预存一些经验数据，再从里面sample一个batch的经验让agent去learn
REWARD_SCALE = 0.01  # reward 的缩放因子
BATCH_SIZE = 256  # 每次给agent learn的数据数量，从replay memory随机里sample一批数据出来
TRAIN_TOTAL_STEPS = 1e3  # 总训练步数
TEST_EVERY_STEPS = 1e2  # 每个N步评估一下算法效果，每次评估5个episode求平均reward
EPISODE_LENGTH = 100 

max_action = float(env.action_space.high[0])
model = RLBenchModel(act_dim, max_action)
algorithm = TD3(model, max_action=max_action, 
                gamma=GAMMA, tau=TAU,actor_lr=ACTOR_LR,
                critic_lr=CRITIC_LR)
agent = RLBenchAgent(algorithm, obs_dim, act_dim)

[06-30 14:40:24 MainThread @machine_info.py:86] nvidia-smi -L found gpu count: 1
[06-30 14:40:24 MainThread @machine_info.py:86] nvidia-smi -L found gpu count: 1
[06-30 14:40:25 MainThread @machine_info.py:86] nvidia-smi -L found gpu count: 1


In [4]:
action = env.action_space.sample()

In [5]:
obs = env.reset()

batch_obs = np.expand_dims(obs, axis=0)
action = agent.predict(batch_obs.astype('float32'))
action = np.squeeze(action)
action = np.clip(action, -1.0, 1.0)  ## special
action = action_mapping(action, env.action_space.low[0],
                        env.action_space.high[0])
# action = np.clip(action, -1.0, 1.0) ## special

next_obs, reward, done, info = env.step(action)

obs = next_obs

2020-06-30 14:40:31,188-INFO: Resetting task: reach_target


In [18]:
obs

array([ 0.00000000e+00,  8.61668587e-01,  1.01504743e+00,  7.15665817e-01,
        9.65768099e-01, -9.94429588e-01, -9.83510017e-01,  8.07857513e-01,
        2.92870522e-01,  5.28673410e-01,  2.50151187e-01, -5.35631299e-01,
       -3.47764134e-01,  8.91275883e-01,  1.06800687e+00,  1.42592704e+00,
        2.85433159e+01,  3.48670512e-01, -1.51665516e+01, -1.44921446e+00,
        2.58828354e+00, -6.17432714e-01,  2.78163165e-01,  1.64687172e-01,
        1.37505448e+00, -1.37740225e-02,  9.95742142e-01, -8.98220316e-02,
       -1.54926321e-02,  2.28568912e-04,  2.22265720e-04,  1.78581476e-03,
        1.38393050e-04, -3.53893643e-04,  1.78134139e-03, -1.39501688e-04,
        4.34511894e-04,  2.92089194e-01,  1.13845184e-01,  1.06613219e+00])

In [ ]:
# RLBench gym env low dim data defination
def get_low_dim_data(self) -> np.ndarray:
    """Gets a 1D array of all the low-dimensional obseervations.
    :return: 1D array of observations.
    """
    low_dim_data = [] if self.gripper_open is None else [[self.gripper_open]]
    for data in [self.joint_velocities, self.joint_positions,
                 self.joint_forces,
                 self.gripper_pose, self.gripper_joint_positions,
                 self.gripper_touch_forces, self.task_low_dim_state]:
        if data is not None:
            low_dim_data.append(data)
    return np.concatenate(low_dim_data)

### Action defination

act_dim is `8`

action `1-7` : joint positons

action `8` : gripper open state `0`:close, `1`:open



In [7]:
action = np.array([0,0,0,0,0,0,0,0])
obs = env.step(action)
obs

(array([ 0.00000000e+00,  2.81333923e-04, -2.38418579e-05,  7.05718994e-04,
        -1.74522400e-03, -9.53674316e-05, -7.00950623e-04,  4.05311584e-04,
        -1.36225186e-02,  1.57914191e-01, -4.78179120e-02, -8.57433259e-01,
         2.96059921e-02,  1.20091605e+00,  7.59181142e-01,  6.67586476e-02,
         2.29018822e+01,  1.38047203e-01, -1.56512051e+01, -2.27602080e-01,
        -4.35375500e+00,  2.23955810e-02,  2.65892804e-01, -3.06994114e-02,
         1.48458481e+00,  9.63596348e-03,  9.91829574e-01,  3.73415253e-03,
         1.27151087e-01,  2.42725015e-04,  2.11358070e-04,  1.77017599e-03,
        -4.61387623e-04,  5.34310930e-05,  1.76935492e-03,  4.61299729e-04,
         2.80320637e-05,  3.81695449e-01, -2.99938887e-01,  8.81210804e-01]),
 0,
 False,
 {})

### Observation defination

obs_dim is `40`

gripper state `1`,   joint_velocities `7`,            

joint positons `7`,  joint_foces `7`,     

gripper_pose `7`,    gripper_joint_positions `2`,  

gripper_touch_forces `6`, task_low_dim_state `3`

gripper pisition is `obs[22:25]`
joint position is `obs[8:15]`

In [59]:
obs = env.reset()
obs

2020-06-29 18:01:25,941-INFO: Resetting task: reach_target


array([ 1.00000000e+00, -1.90734863e-05, -5.68866730e-03, -5.72204590e-05,
        1.06811523e-03, -1.19209290e-04, -1.32560730e-03, -3.81469727e-05,
       -2.61902642e-06,  1.75681859e-01, -7.35792491e-06, -8.73253047e-01,
       -4.16721196e-06,  1.22111416e+00,  7.85380244e-01, -4.65409942e-02,
        2.39213181e+01, -6.12235256e-02, -1.62041206e+01, -1.59019470e-01,
       -4.32092381e+00,  3.63271907e-02,  2.78860390e-01, -8.15512240e-03,
        1.47118556e+00, -1.83602128e-06,  9.92758512e-01, -1.25260358e-05,
        1.20127596e-01,  3.99997607e-02,  3.99997421e-02,  1.78868475e-03,
       -4.28783882e-04,  2.40951772e-06,  1.78653153e-03,  4.28555912e-04,
       -1.87903640e-06,  2.02118695e-01, -5.70433326e-02,  1.16482604e+00])

In [42]:
# joint position is obs[8:15]
obs[8:15]

array([ 1.43408943e-06,  1.75640374e-01,  2.17881734e-06, -8.73171985e-01,
       -1.54460577e-06,  1.22106075e+00,  7.85394311e-01])

In [60]:
## gripper positon is obs[29:31], one of them
obs[29:31]

array([0.03999976, 0.03999974])

In [43]:
# target position is obs[-3:]
obs[-3:]

array([ 0.36140263, -0.30578855,  1.19540811])

In [44]:
action = env.action_space.sample()

In [45]:
next_obs, reward, done, info = env.step(action)

In [46]:
next_obs

array([ 1.00000000e+00,  3.99408340e-01,  4.87675667e-01,  7.24291801e-01,
       -7.26757050e-01, -5.73062897e-01,  8.14661980e-01,  2.52575874e-01,
        1.99718513e-02,  2.00024158e-01,  3.62167694e-02, -9.09509838e-01,
       -2.86546890e-02,  1.26179385e+00,  7.98023105e-01, -4.86057739e+01,
       -4.71883278e+01, -4.06475906e+01, -2.79576359e+01, -1.20000000e+01,
        5.60368824e+00, -1.20000000e+01,  2.95849860e-01,  1.03620701e-02,
        1.45198393e+00, -1.20479614e-02,  9.93561447e-01, -5.00657829e-03,
        1.12541355e-01,  3.99968848e-02,  3.99717838e-02, -1.02909503e-03,
       -1.60410255e-03,  2.18392769e-03, -9.61102545e-04,  1.64184871e-03,
       -2.18251650e-03,  3.61402631e-01, -3.05788547e-01,  1.19540811e+00])

In [61]:
env.observation_space.shape[0]

40

In [62]:
env.action_space.shape[0]

8